In [36]:
from HelperInfra import get_target_date , last_friday_of_previous_month , last_thursday ,get_data
import pandas as pd
from datetime import datetime

In [37]:
stock_price=pd.read_csv('../All Close Prices.csv')
stock_price['Date'] = pd.to_datetime(stock_price['Date']).dt.date
stock_price=stock_price[stock_price['Exchange']=='NSE']
stock_price=stock_price.sort_values(by='Date',ascending=True)

In [38]:

stock_price

,Name,Date,Close,Exchange,Year
7139800,Hybrid Financial Services Ltd.,2001-04-01,2.350,NSE,2001
4647007,EPL Ltd.,2001-04-01,21.285,NSE,2001
197096,AMJ Land Holdings Ltd.,2001-04-01,3.110,NSE,2001
12700823,PCBL Ltd.,2001-04-01,1.600,NSE,2001
16974302,Sri Havisha Hospitality and Infrastructure Ltd.,2001-04-01,0.500,NSE,2001
...,...,...,...,...,...
15923862,Semac Consultants Ltd.,2023-12-21,2376.500,NSE,2023
10259832,Mahanagar Telephone Nigam Ltd.,2023-12-21,32.400,NSE,2023
7252246,IFB Agro Industries Ltd.,2023-12-21,470.550,NSE,2023
14289430,Rane Holdings Ltd.,2023-12-21,1193.300,NSE,2023


In [39]:
import pandas as pd
database=pd.read_csv('./FinalDataBase.csv')
database
database["Start Date"] = pd.to_datetime(database["Start Date"], format="%d-%m-%Y")
database["End Date"] = pd.to_datetime(database["End Date"], format="%d-%m-%Y")
#Getting The Database Which has The Mapping Of Symbol-Company Name-Accord Code

In [40]:
database

,Unnamed: 0,Stock,Start Date,End Date,Accord Code,Sr.No.,Company Name,CD_BSE Code,CD_Bse Scrip Name,CD_NSE Symbol,CD_ISIN No
0,0,ABB,2006-12-29,2013-02-01,100002,33,ABB India Ltd.,500002.0,ABB LTD.,ABB,INE117A01022
1,1,SUZLON,2006-12-29,2013-03-01,132667,3504,Suzlon Energy Ltd.,532667.0,SUZLONENERGY,SUZLON,INE040H01021
2,2,GUJFLUORO,2011-08-05,2013-03-01,295872,1278,Gujarat Fluorochemicals Ltd.,542812.0,FLUOROCHEM,FLUOROCHEM,INE09N301011
3,3,GVKPIL,2008-08-21,2013-04-01,132708,1298,GVK Power & Infrastructure Ltd.,532708.0,GVK POWERINF,GVKPIL,INE251H01024
4,4,BHUSANSTL,2007-09-05,2013-04-01,100055,3571,Tata Steel BSL Ltd.,500055.0,TATASTLBSL,TATASTLBSL,INE824B01021
...,...,...,...,...,...,...,...,...,...,...,...
314,314,LT,2006-12-29,2024-12-31,100510,1944,Larsen & Toubro Ltd.,500510.0,LARSEN & TOU,LT,INE018A01030
315,315,LUPIN,2006-12-29,2024-12-31,100257,2002,Lupin Ltd.,500257.0,LUPIN LTD.,LUPIN,INE326A01037
316,316,M_M,2006-12-29,2024-12-31,100520,2041,Mahindra & Mahindra Ltd.,500520.0,MAH & MAH,M&M,INE101A01026
317,317,RBLBANK,2017-05-26,2024-12-31,209068,2949,RBL Bank Ltd.,540065.0,RBLBANK,RBLBANK,INE976G01028


In [41]:
valid_stocks=pd.read_csv('./StockValidation.csv')
valid_stocks["Date"] = pd.to_datetime(valid_stocks["Date"]).dt.date

valid_stocks

,Unnamed: 0,Date,Symbol
0,0,2013-01-01,ABB
1,1,2013-01-01,SUZLON
2,2,2013-01-01,GUJFLUORO
3,3,2013-01-01,GVKPIL
4,4,2013-01-01,BHUSANSTL
...,...,...,...
3344,3344,2022-07-01,LICHSGFIN
3345,3345,2022-07-01,LT
3346,3346,2022-07-01,LUPIN
3347,3347,2022-07-01,M_M


In [42]:
len(valid_stocks)

3349

In [43]:
stock_price=stock_price[(stock_price['Date']>=datetime(2012,1,1).date()) & (stock_price['Date']<=datetime(2023,1,1).date())]

In [20]:
import pandas as pd
from datetime import datetime

# Pre-compute `Company Name_y` mapping
symbol_to_company = database.set_index('Stock')['Company Name'].to_dict()

# Pre-group stock_price by company name
stock_price_grouped = {name: group for name, group in stock_price.groupby('Name')}

# Define function to calculate percentage change
def find_start_end_close(row):
    year = row['Date'].year
    month = row['Date'].month
    
    # Get date ranges
    one_year_earlier = get_target_date(datetime(year, month, 1), '-12F', 2).date()
    current_date = get_target_date(datetime(year, month, 1), '0F', 2).date()
    
    # Get company name
    company_name = symbol_to_company.get(row['Symbol'])
    if company_name is None:
        return None, None, None, None, None  # If no mapping found
    
    # Get stock price data for this company
    stock_price_changed = stock_price_grouped.get(company_name)
    if stock_price_changed is None:
        return None, None, None, None, None  # If no stock data found
    
    # Filter relevant rows
    close_price = stock_price_changed[
        (one_year_earlier <= stock_price_changed['Date']) & 
        (stock_price_changed['Date'] <= current_date)
    ]
    
    if not close_price.empty:
        prev_price = close_price['Close'].iloc[0]
        curr_price = close_price['Close'].iloc[-1]
        percentage = (curr_price - prev_price) / prev_price
        return close_price['Date'].iloc[0], curr_price, close_price['Date'].iloc[-1], prev_price, percentage
    else:
        return None, None, None, None, None

# Apply the optimized function to the DataFrame
valid_stocks[['Prev Date', 'Curr Price', 'Curr Date', 'Prev Price', '6 Months Returns']] = valid_stocks.apply(
    find_start_end_close,
    axis=1,
    result_type='expand'
)


valid_stocks = valid_stocks.dropna()
valid_stocks.dropna(inplace=True)
from datetime import timedelta
valid_stocks['Diff'] = (valid_stocks['Curr Date'] - valid_stocks['Prev Date'])
valid_stocks=valid_stocks[valid_stocks['Diff']>=timedelta(days=365)]
valid_stocks.sort_values(by='Diff')

/tmp/ipykernel_1596/200817227.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_stocks.dropna(inplace=True)
/tmp/ipykernel_1596/200817227.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_stocks['Diff'] = (valid_stocks['Curr Date'] - valid_stocks['Prev Date'])


,Unnamed: 0,Date,Symbol,Prev Date,Curr Price,Curr Date,Prev Price,6 Months Returns,Diff
1694,1694,2018-01-01,RELIANCE,2017-01-01,901.207160,2018-01-01,536.117961,0.680987,"365 days, 0:00:00"
2053,2053,2019-01-01,ARVIND,2018-01-01,98.400000,2019-01-01,445.950000,-0.779347,"365 days, 0:00:00"
2054,2054,2019-01-01,OFSS,2018-01-01,3729.450000,2019-01-01,4066.750000,-0.082941,"365 days, 0:00:00"
2055,2055,2019-01-01,IDBI,2018-01-01,61.450000,2019-01-01,60.350000,0.018227,"365 days, 0:00:00"
2056,2056,2019-01-01,MCX,2018-01-01,734.950000,2019-01-01,920.050000,-0.201185,"365 days, 0:00:00"
...,...,...,...,...,...,...,...,...,...
2740,2740,2021-01-01,BANKBARODA,2020-01-01,64.500000,2021-01-01,101.900000,-0.367026,"366 days, 0:00:00"
2741,2741,2021-01-01,BATAINDIA,2020-01-01,1573.900000,2021-01-01,1745.300000,-0.098207,"366 days, 0:00:00"
2742,2742,2021-01-01,BEL,2020-01-01,42.133333,2021-01-01,33.383333,0.262107,"366 days, 0:00:00"
2731,2731,2021-01-01,APOLLOTYRE,2020-01-01,179.900000,2021-01-01,163.350000,0.101316,"366 days, 0:00:00"


In [21]:
def calculate_std(row):
    # Get company name
    company_name = symbol_to_company.get(row['Symbol'])
    if company_name is None:
        return None
    
    # Get stock price data for this company
    stock_price_changed = stock_price_grouped.get(company_name)
    filtered_data = stock_price_changed.sort_values(by='Date', ascending=True)
    
    # Calculate percentage change and standard deviation
    filtered_data['Pct_Change'] = filtered_data['Close'].pct_change()
    std_dev = filtered_data['Pct_Change'].std()
    
    return std_dev

# Apply the function without result_type='expand'
valid_stocks['STD'] = valid_stocks.apply(calculate_std, axis=1)


In [22]:
valid_stocks['Ratio']=valid_stocks['6 Months Returns']/valid_stocks['STD']

In [23]:
valid_stocks

,Unnamed: 0,Date,Symbol,Prev Date,Curr Price,Curr Date,Prev Price,6 Months Returns,Diff,STD,Ratio
0,0,2013-01-01,ABB,2012-01-01,705.500000,2013-01-01,583.300000,0.209498,"366 days, 0:00:00",0.017077,12.267788
1,1,2013-01-01,SUZLON,2012-01-01,17.262423,2013-01-01,16.527851,0.044444,"366 days, 0:00:00",0.032337,1.374432
3,3,2013-01-01,GVKPIL,2012-01-01,14.150000,2013-01-01,11.550000,0.225108,"366 days, 0:00:00",0.035038,6.424692
5,5,2013-01-01,WELCORP,2012-01-01,106.100000,2013-01-01,84.100000,0.261593,"366 days, 0:00:00",0.027307,9.579851
6,6,2013-01-01,IVRCLINFRA,2012-01-01,46.550000,2013-01-01,28.250000,0.647788,"366 days, 0:00:00",0.031697,20.436590
...,...,...,...,...,...,...,...,...,...,...,...
3344,3344,2022-07-01,LICHSGFIN,2021-07-01,338.250000,2022-07-01,470.250000,-0.280702,"365 days, 0:00:00",0.019166,-14.645566
3345,3345,2022-07-01,LT,2021-07-01,1572.150000,2022-07-01,1492.300000,0.053508,"365 days, 0:00:00",0.015189,3.522738
3346,3346,2022-07-01,LUPIN,2021-07-01,628.300000,2022-07-01,1146.200000,-0.451841,"365 days, 0:00:00",0.015335,-29.465391
3347,3347,2022-07-01,M_M,2021-07-01,1107.350000,2022-07-01,779.450000,0.420681,"365 days, 0:00:00",0.015751,26.707697


In [24]:
# Group by 'Date' and sort by 'Ratio' within each group, keeping the top 30 rows
top_30_by_date = valid_stocks.groupby('Date').apply(lambda x: x.sort_values(by='Ratio', ascending=False).head(30))

# Reset the index to maintain a clean DataFrame
top_30_by_date = top_30_by_date.reset_index(drop=True)

/tmp/ipykernel_1596/3198476709.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_30_by_date = valid_stocks.groupby('Date').apply(lambda x: x.sort_values(by='Ratio', ascending=False).head(30))


In [25]:
top_30_by_date.to_csv('Top 30 Stock List For Momemtum.csv')

In [26]:
import pandas as pd
from datetime import datetime
from collections import defaultdict
from datetime import datetime, timedelta
import calendar
import numpy as np
import pandas as pd
def last_friday_of_previous_month(year, month):
    if month == 1:
        year -= 1
        month = 12
    else:
        month -= 1

    last_day = calendar.monthrange(year, month)[1]
    last_date = datetime(year, month, last_day)

    offset = (last_date.weekday() - 3) % 7  # 3 = Thursday
    last_thursday_date = last_date - timedelta(days=offset)
    return (last_thursday_date + timedelta(days=1)).date()  # Adjusting to Friday if needed

def last_thursday(year, month):
    last_day = calendar.monthrange(year, month)[1]
    last_date = datetime(year, month, last_day)

    offset = (last_date.weekday() - 3) % 7  # 3 = Thursday
    last_thursday_date = last_date - timedelta(days=offset)
    return last_thursday_date.date()

In [27]:
print(last_friday_of_previous_month(2024,11)," ",last_thursday(2024,11))

2024-11-01   2024-11-28


In [28]:
global_arr = []
for year in range(2013, 2023):
    for month in range(1, 13):
        correct_month = 1 if month <= 6 else 7
        current_date = datetime(year, correct_month, 1).date()  # Ensure current_date is a date object
        stock_list = top_30_by_date[top_30_by_date['Date'] == current_date]
        
        # Get last Friday and Thursday of the previous month
        lf = last_friday_of_previous_month(year, month)
        lt = last_thursday(year, month)
        
        tell = False
        
        for _, stock in stock_list.iterrows():
            stock_details = database[database['Stock'] == stock['Symbol']]
            
            for _, details in stock_details.iterrows():
                # Ensure start and end dates are datetime objects to allow comparison
                start_date = details['Start Date'] if isinstance(details['Start Date'], datetime) else pd.to_datetime(details['Start Date'])
                end_date = details['End Date'] if isinstance(details['End Date'], datetime) else pd.to_datetime(details['End Date'])
                
                # Compare dates correctly
                # print(start_date.date(), current_date, end_date.date())
                if (start_date.date() <= current_date) and (current_date <= end_date.date()):
                    tell = True

            if tell:
                global_arr.append({
                    'Symbol': stock['Symbol'],  # Append the stock symbol, not the entire stock Series
                    'Start': lf,
                    'End': lt
                })
                tell = False  
            else :
                print(stock)
                


In [29]:
global_arr
df=pd.DataFrame(global_arr)

In [32]:
def find_accord_code(row):
    info=database[database['Stock']==row['Symbol']]
    return info.iloc[0]['Company Name'],info.iloc[0]['Accord Code']

In [33]:
df[['Company Name','Accord Code']] = df.apply(
    find_accord_code,
    axis=1,
    result_type='expand'
)

In [34]:
df

,Symbol,Start,End,Company Name,Accord Code
0,UNITDSPR,2012-12-28,2013-01-31,United Spirits Ltd.,132432
1,KTKBANK,2012-12-28,2013-01-31,The Karnataka Bank Ltd.,132652
2,DENABANK,2012-12-28,2013-01-31,Dena Bank - (Amalgamated),132121
3,AUROPHARMA,2012-12-28,2013-01-31,Aurobindo Pharma Ltd.,124804
4,MRF,2012-12-28,2013-01-31,MRF Ltd.,100290
...,...,...,...,...,...
3595,ESCORTS,2022-11-25,2022-12-29,Escorts Kubota Ltd.,100495
3596,CUMMINSIND,2022-11-25,2022-12-29,Cummins India Ltd.,100480
3597,UNITDSPR,2022-11-25,2022-12-29,United Spirits Ltd.,132432
3598,TATAMOTORS,2022-11-25,2022-12-29,Tata Motors Ltd.,100570


In [35]:
df.to_csv('MomentumUniverseForEachMonth.csv')